In [1]:
import random
import numpy as np
import pandas as pd
from scipy import stats
train = pd.read_csv('hw1data/propublicaTrain.csv')
test = pd.read_csv('hw1data/propublicaTest.csv')

In [9]:
#Training-Validation Split
raw_train_labels=np.array(train['two_year_recid'])
raw_train_input = np.array([np.array(train['sex']),np.array(train['age']),np.array(train['race']),np.array(train['juv_fel_count']),np.array(train['juv_misd_count']),np.array(train['juv_other_count']),np.array(train['priors_count']),np.array(train['c_charge_degree_F'])])
validation_pct = 20
val_size = validation_pct*len(raw_train_labels)//100
val_ind = random.sample(range(len(raw_train_labels)), val_size)
validation_input = np.zeros((8,val_size))
validation_labels = np.zeros(val_size)
training_input = np.zeros((8,len(raw_train_labels)-val_size))
training_labels = np.zeros(len(raw_train_labels)-val_size)
cat_names = ['sex','age','race','juv_fel_count','juv_misd_count','juv_other_count','priors_count','c_charge_degree_F']
val_cnt = 0
trn_cnt = 0
for i in range(len(raw_train_labels)):
    if i in val_ind:
        for j in range(len(cat_names)):
            validation_input[j,val_cnt]=train[cat_names[j]][i]
        validation_labels[val_cnt]=train['two_year_recid'][i]
        val_cnt += 1
    else:
        for j in range(len(cat_names)):
            training_input[j,trn_cnt]=train[cat_names[j]][i]
        training_labels[trn_cnt]=train['two_year_recid'][i]
        trn_cnt += 1
val_cnt, trn_cnt

(833, 3334)

In [3]:
#Normed Distance
def distance(x,y,p):
    if (p=='inf'):
        all_dist = np.zeros(np.shape(x)[0])
        for i in range(np.shape(x)[0]):
            all_dist[i] = abs(x[i]-y[i])
        return np.amax(all_dist)
    else:
        sum = 0
        for i in range(np.shape(x)[0]):
            sum += (abs(x[i]-y[i]))**p
        return sum**(1/p)


In [4]:
#Reading in Test Data
test_labels = np.array(test['two_year_recid'])
test_input = np.array([np.array(test['sex']),np.array(test['age']),np.array(test['race']),np.array(test['juv_fel_count']),np.array(test['juv_misd_count']),np.array(test['juv_other_count']),np.array(test['priors_count']),np.array(test['c_charge_degree_F'])])

In [ ]:
#Unweighted k-NN Classification on Validation Data
k_array = [k for k in range(1,16,2)]
p_array = [1,2,3,'inf']
for k in k_array:
    for p in p_array:
        pred_validation_labels = np.zeros(len(validation_labels))
        for i in range(len(validation_labels)):
            all_distances = np.zeros(len(training_labels))
            for j in range(len(training_labels)):
                all_distances[j]=distance(validation_input[:,i],training_input[:,j],p)
            ind = np.argpartition(all_distances,k)[:k]
            nearest_labels = [training_labels[temp] for temp in ind]
            pred_validation_labels[i] = stats.mode(nearest_labels, axis=None)[0][0]
        num_correct = 0
        for ent in range(len(validation_labels)):
            if (validation_labels[ent]==pred_validation_labels[ent]):
                num_correct += 1
        #print(validation_labels)
        #print(pred_validation_labels)
        print("k="+str(k) + ",p=" + str(p) + ": " + str(num_correct/len(validation_labels)*100))

In [ ]:
#Normalized [-1,1] kNN Classification on Validation Data
norm_training_input = np.zeros((8,len(training_labels)))
norm_validation_input = np.zeros((8,len(validation_labels)))
norm_test_input = np.zeros((8,len(test_labels)))
cat_num = [0,2,7]
#Normalization
for j in range(len(cat_names)):
    x = np.amax(training_input[j,:])
    for cnt0 in range(len(training_labels)):
        if j in cat_num:
            norm_training_input[j,cnt0] = -1 + 2*training_input[j,cnt0]
        else:
            norm_training_input[j,cnt0] = -1 + 2*training_input[j,cnt0]/x
    for cnt1 in range(len(validation_labels)):
        if j in cat_num:
            norm_validation_input[j,cnt1] = -1 + 2*validation_input[j,cnt1]
        else:
            norm_validation_input[j,cnt1] = -1 + 2*validation_input[j,cnt1]/x
    for cnt2 in range(len(test_labels)):
        if j in cat_num:
            norm_test_input[j,cnt2] = -1 + 2*test_input[j,cnt2]
        else:
            norm_test_input[j,cnt2] = -1 + 2*test_input[j,cnt2]/x
#kNN
for k in k_array:
    for p in p_array:
        pred_norm_validation_labels = np.zeros(len(validation_labels))
        for i in range(len(validation_labels)):
            all_distances = np.zeros(len(training_labels))
            for j in range(len(training_labels)):
                all_distances[j]=distance(norm_validation_input[:,i],norm_training_input[:,j],p)
            ind = np.argpartition(all_distances,k)[:k]
            nearest_labels = [training_labels[temp] for temp in ind]
            pred_norm_validation_labels[i] = stats.mode(nearest_labels, axis=None)[0][0]
        num_correct = 0
        for ent in range(len(validation_labels)):
            if (validation_labels[ent]==pred_norm_validation_labels[ent]):
                num_correct += 1
        #print(validation_labels)
        #print(pred_validation_labels)
        print("NORM; k="+str(k) + ",p=" + str(p) + ": " + str(num_correct/len(validation_labels)*100))

In [6]:
#Running the best model, based on validation data, on our testing data
k_final=15
p_final=1
pred_test_labels = np.zeros(len(test_labels))
for i in range(len(test_labels)):
    all_distances = np.zeros(len(training_labels))
    for j in range(len(training_labels)):
        all_distances[j]=distance(test_input[:,i],training_input[:,j],p_final)
    ind = np.argpartition(all_distances,k_final)[:k_final]
    nearest_labels = [training_labels[temp] for temp in ind]
    pred_test_labels[i] = stats.mode(nearest_labels, axis=None)[0][0]
num_correct = 0
for ent in range(len(test_labels)):
    if (test_labels[ent]==pred_test_labels[ent]):
        num_correct += 1
ACCURACY = num_correct/len(test_labels)*100
TEST_PRED_LIST = pred_test_labels
print(ACCURACY)

67.4


In [ ]:
#Training and testing data dataframes as parameters, returns accuracy,testing_predictions